# Specifications
Objective:

* 理解函数规范中的前置条件(preconditions)和后置条件(postconditions)，并能够编写正确的规范
* 能够根据规范编写测试
* 了解如何使用异常

规范是团队合作的关键。如果没有规范，就不可能将实现功能的责任委托给他人。规范就像一份合同：实现者有责任履行合同，而使用该功能的客户可以信赖合同。事实上，我们将看到，就像真正的法律合同一样，规范对双方都提出了要求：当规范有先决条件时，客户也有责任完成条件。

在本章中，我们将探讨函数规范所扮演的角色。我们将讨论什么是前置条件和后置条件，以及它们对函数的实现者和客户意味着什么。我们还将讨论如何使用异常，异常是一种重要的语言特性，不仅存在于TypeScript中，也存在于Python、Java和许多其他现代语言中。

## Behavioral equivalence
假设正在编写寻找在array的=val的索引值
```ts
function find(arr: Array<number>, val: number): number {
    for (let i = 0; i < arr.length; i++) {
        if (arr[i] === val) return i;
    }
    return -1;
}
```
如果在一个大的array中，你发现查找的值大概率要么在头部，要么在尾部，于是

```ts
function find(arr: Array<number>, val: number): number {
    for (let i = 0, j = arr.length-1; i <= j; i++, j--) {
        if (arr[i] === val) return i;
        if (arr[j] === val) return j;
    }
    return -1;
}
```

用这个新的实现替换 find 是否安全？我们是否可以在不引入错误的情况下进行这种改变？为了确定行为等价性，我们询问是否可以在不影响正确性的情况下用一种实现替代另一种实现。

这些实现不仅具有不同的性能特征，而且对于某些输入，它们实际上具有不同的输出行为。如果val在数组中出现不止一次，原始的find总是返回它出现的最低索引。但新的查找可能返回最低索引或最高索引，以先找到的索引为准。

然而，当val正好只出现在数组的一个索引中时，两种实现的行为是相同的：它们都返回该索引。在这种情况之外，客户端可能从不依赖这种行为。每当他们调用函数时，都会传入一个数组，该数组中正好有一个元素与val匹配。对于这样的客户端，这两个版本的find是相同的，我们可以从一个实现切换到另一个实现而不会出现问题。

行为等价的概念是在观察者--也就是站在客户端的角度。为了能够用一种实现替代另一种实现，并且知道什么时候可以接受这种替代，我们需要一个能够准确说明客户端依赖于什么的规范。

在这种情况下，允许这两种实现在行为上等价的规范可以是：
```ts
find(arr: Array<number>, val: number): number
requires:
val occurs exactly once in arr
effects:
returns index i such that arr[i] = val
```




## Why specifications?

我们找到的例子展示了规范如何帮助程序既能适应变化又能避免错误。程序中许多最糟糕的bug都是由于对两段代码之间接口行为的误解而产生的。尽管每个程序员心中都有规范，但并非所有程序员都会将其写下来。因此，一个团队中的不同程序员可能会有不同的规范。当程序出现故障时，很难确定错误出在哪里。代码中的精确规范可以让您分清责任（是代码片段的责任，而不是人的责任！），并且可以让您不必为错误修复的位置而苦恼。

规范对模块的客户端来说是件好事，因为它们有助于模块更容易理解，就像黑盒子外面的标签一样。有了规范，你就可以理解模块的功能，而不必去阅读模块的代码。如果您不相信阅读规范比阅读代码更容易，请比较：

```
find(arr: Array<number>, val: number): number
requires:
val occurs exactly once in arr
effects:
returns index i such that arr[i] = val
```

```ts
function find(arr: Array<number>, val: number): number {
    for (let i = 0, j = arr.length-1; i <= j; i++, j--) {
        if (arr[i] === val) return i;
        if (arr[j] === val) return j;
    }
    return -1;
}

```


规范对于函数的实现者来说是件好事，因为它给了实现者在不告诉客户端的情况下改变实现的自由。规范还可以使代码更快。我们将看到，规范可以排除函数可能被调用的某些状态。限制输入可以让实现者跳过不再需要的昂贵检查，并使用更高效的实现。我们将在后面的阅读中看到这样一个例子。

契约就像客户机和实现者之间的防火墙。它使客户免受模块工作细节的影响：作为客户，如果你有模块的规范，你就不需要阅读模块的源代码。它使实现者免受模块使用细节的影响：作为实现者，你不必询问每个客户打算如何使用模块。这道防火墙的结果是解耦，允许模块的代码和客户端的代码独立地被修改，只要这些修改遵守了规范--各自都遵守合同中的义务。
![](./ref/lect4/2023-07-17-17-03-59.png)

## Specification structure

抽象地讲，一个函数的规范包括几个部分：

* 函数签名，给出名称、参数类型和返回类型
* required子句，描述对参数的附加限制
* 效果子句，描述函数的返回值、异常和其他效果。


这些部分共同构成了函数的前置条件和后置条件。

前置条件是客户（函数的调用者）的义务。它是函数被调用时所处状态的条件。前提条件的一个方面是函数签名中参数的数量和类型，TypeScript可以静态检查。其他条件写在required子句中，例如

* 缩小参数类型（例如，x是一个整数>= 0，表示数字参数x实际上必须是一个非负整数）
* 参数之间的交互（例如，val在arr中恰好出现一次）

后置条件是函数实现者的义务。TypeScript可以静态检查后置条件的某些部分，特别是返回类型。其他条件写在effects子句中，包括

* 返回值与输入的关系。
* 何时抛出何种异常
* object是否发生变化以及如何变化

一般来说，后置条件是函数调用后程序状态的条件，假定前置条件为真。

整体结构是一个逻辑蕴涵（logical implication）：如果在调用函数时前提条件成立，那么在函数完成时后条件必须成立。
![](./ref/lect4/2023-07-17-17-00-40.png)

如果前置条件在函数调用时不成立，那么实现不受后置条件的约束。它可以自由地做任何事情，包括永不返回、抛出规范中未提及的异常、返回任意结果、进行任意突变等。
![](./ref/lect4/2023-07-17-17-02-23.png)


## Specifications in TypeScript

一些语言（特别是Eiffel）将前置条件和后置条件作为语言的基本组成部分，作为运行时系统（甚至编译器）可以自动检查的表达式，以执行客户端和实现者之间的契约。

TypeScript没有走得这么远，但是它的静态类型声明实际上是函数的前置条件和后置条件的一部分，这部分由编译器自动检查和执行。契约的其余部分--我们不能写成类型的部分--必须在函数前面的注释中描述，并且通常要靠人来检查和保证。

一个常见的约定，最初是为Java设计的，但现在也被TypeScript和JavaScript使用，就是在函数前加上文档注释，其中

* 参数由@param子句描述
* 结果由@returns子句描述

前置条件会放在@param，后置条件会放在@param。

```ts
find(arr: Array<number>, val: number): number
requires:
val occurs exactly once in arr
effects:
returns index i such that arr[i] = val
```
TypeDoc 如下格式：
```ts
/**
 * Find a value in an array.
 * @param arr array to search, requires that val occurs exactly once
 *            in arr
 * @param val value to search for
 * @returns index i such that arr[i] = val
 */
function find(arr: Array<number>, val: number): number
```
[generate HTML documentation automatically](https://typedoc.org/)

### Reading exercise
TypeDoc
Given this spec:

isPalindrome(word: string): boolean
requires:
word contains only alphanumeric characters
effects:
returns true if and only if word is a palindrome
Here is a flawed attempt to write the spec in TypeDoc:

```
Which lines in the TypeDoc comment are problematic?
/*
 * Check if a word is a palindrome.
 * A palindrome is a sequence of characters
 * that reads the same forwards and backwards.
 * @param string word
 * @requires word contains only alphanumeric characters
 * @effects returns true if and only if word is a palindrome
 * @returns boolean
 */
```

The start of the comment is wrong: /* should be /** in order to be interpreted as TypeDoc rather than an ordinary comment.

@param string word should not include the type string (the function signature already specifies the type), and should include some description of what the word parameter means – and particularly the precondition currently found in the @requires line.

@requires and @effects are not TypeDoc syntax; preconditions and postconditions should be written in clauses like @param and @returns.

@returns boolean should not include the type, but should include the postcondition.

Here’s a better TypeDoc comment:

/**
 * Check if a word is a palindrome.
 * A palindrome is a sequence of characters
 * that reads the same forwards and backwards.
 * @param word word to check, must contain only alphanumeric characters
 * @returns true if and only if word is a palindrome
 */

### What a specification may talk about

函数的规范可以谈论函数的参数和返回值，但绝不能谈论函数的局部变量或函数类的私有字段。您应该将实现视为对规范读者不可见。就客户端而言，它处于防火墙之后。
![](ref/lect4/2023-07-18-10-53-40.png)

函数的源代码甚至可能对规范的读者不可用，因为TypeDoc工具只从您的代码中提取规范注释并将它们渲染为HTML。这种分离是一件好事：规范使客户不必担心（或不适当地依赖于！）函数实现的细节。客户端只需关注规范。


## Avoid null
大多数的语言会添加一个特殊值：None或者null，表示不指向某一个对象,默认情况下，任何类型都可以是使用个特殊值赋值。
```ts
let word: string = null;  // legal by default!
```

这是静态类型系统中一个不幸的漏洞，因为null并不是一个真正合法的字符串值。您不能调用该引用的任何方法或使用任何属性：
```ts
word.toLowerCase() // throws TypeError
word.length        // throws TypeError
```
它与""和[]不同，后者依然可以调用方法，使用length返回的是0。

空值既麻烦又不安全，因此好的程序设计都会尽量避免空值。一般来说，**除非规范明确说明，否则在参数和返回值中不允许出现空值。** 因此，每个函数都有一个前提条件，即其参数中的对象和数组类型必须为非空值，包括数组、集合和映射等集合中的元素。每个可以隐式返回对象或数组类型的函数都有一个后置条件，即它们的值是非空的，同样包括集合类型的元素。

当TypeScript启用了[严格的空检查](https://www.typescriptlang.org/tsconfig#strictNullChecks)（这是6.102的配置），静态类型检查器保证了这一点：
```ts
let word: string = null;  // compile error when strict null checking is on
let words: Array<string> = [ null ]; // compile error when strict null checking is on
```
如果函数允许参数或返回值为null，则需要在类型中明确声明null（例如string|null）。但应尽量避免这样做。避免使用null。

[Google在其核心Java库Guava中对null有自己的论述](https://github.com/google/guava/wiki/UsingAndAvoidingNullExplained)。该项目解释说

Careless use of null can cause a staggering variety of bugs. Studying the Google code base, we found that something like 95% of collections weren’t supposed to have any null values in them, and having those **fail fast** rather than silently accept null would have been helpful to developers.

Additionally, null is unpleasantly ambiguous. It’s rarely obvious what a null return value is supposed to mean — for example, Map.get(key) [in Java] can return null either because the value in the map is null, or the value is not in the map. Null can mean failure, can mean success, can mean almost anything. Using something other than null **makes your meaning clear.**

## Include emptiness

确保了解null和emptiness之间的区别。

回想一下，在 Python 中，None 与空字符串""、空数组 [ ] 或空字典 { } 不同。这些empty objects 是有效的对象，只是碰巧不包含任何元素。但是您可以在类型允许的所有常规操作中使用它们。例如，len("")返回0，"" + "a "返回 "a"。而 None 则不然：len(None) 和 None + "a" 都会产生错误。

同样的想法也可以转换到TypeScript中。null 引用不是一个有效的字符串、数组、map或任何其他对象。但是empty字符串""是一个有效的字符串值，empty 数组[ ]是一个有效的数组值。

The upshot of this is that **empty values are always allowed as parameter or return values** unless a spec explicitly disallows them.

## Testing and specifications

在测试中，我们谈论黑盒测试和玻璃盒测试，黑盒测试只考虑规范，而玻璃盒测试则考虑实际实现（[测试](https://web.mit.edu/6.031/www/sp23/classes/02-testing/#black_box_and_glass_box_testing)）。但需要注意的是，即使是**玻璃箱测试也必须遵循规范**。您的实现可能提供比规范要求更强的保证，或者它可能有规范未定义的特定行为。但是您的测试用例不应该依赖于这种行为。测试用例必须是[正确的](https://web.mit.edu/6.031/www/sp23/classes/02-testing/#systematic_testing)，就像其他客户端一样遵守契约。

例如，假设您正在测试 find 的这个规范，它与我们到目前为止使用的规范略有不同：
```
find(arr: Array<number>, val: number): number
requires: val occurs in arr
effects: returns index i such that arr[i] = val
```
这个规范有一个很强的前置条件，即要求找到 val；它有一个相当弱的后置条件，即如果 val 在数组中出现了不止一次，这个规范没有说要返回 val 的哪个特定索引。即使你实现的 find 总是返回最低的索引，你的测试用例也不能假设这个特定的行为：
```ts
let array: Array = [7, 7, 7];
let i = find(array, 7);
assert.strictEqual(i, 0);  // bad test case: assumes too much, more than the postcondition promises
assert.strictEqual(array[i], 7);  // correct
```
类似地，即使你实现了find，当没有找到val时，它（理智地）抛出异常，而不是返回一些任意的误导性索引，你的测试用例也不能假设这种行为，因为它不能以违反先决条件的方式调用find()。

### Testing units
```ts
/** 
 * @returns the contents of the file
 */
function load(filename: string): string { ... }

/** 
 * @returns the words in string s, in the order they appear,
 *         where a word is a contiguous sequence of
 *         non-whitespace and non-punctuation characters
 */
function extract(s: string): Array<string> { ... }

/**
 * @returns an index mapping a word to the set of filenames
 *         containing that word, for all files in the input set
 */
function index(filenames: Set<string>): Map<string, Set<string>>  {
    ...
    for (let file of files) {
        let doc = load(file);
        let words = extract(doc);
        ...
    }
    ...
} 
```

我们谈到了单元测试，即我们应该对程序的每个模块单独编写测试。一个好的单元测试只关注一个规范。我们的测试几乎总是依赖于JavaScript库函数的规范，但是如果我们编写的一个函数的单元测试不能满足另一个函数的规范，那么这个函数的单元测试就不应该失败。正如我们在示例中看到的，如果load()的后置条件不满足，那么对extract()的测试就不应该失败。

好的集成测试，即使用模块组合的测试，将确保我们的不同函数具有兼容的规范：不同函数的调用者和实现者按照对方的期望传递和返回值。集成测试不能取代系统设计的单元测试。从示例中可以看出，如果我们只通过调用index来测试extract，那么我们将只在其输入空间的一小部分上测试它：输入是load的可能输出。这样做，我们就为bug留下了藏身之处，当我们在程序的其他地方为不同的目的使用extract时，或者当load开始返回以新格式编写的文档时，bug就会跳出来。

### All tests must follow the spec

We cannot test for behavior when a precondition is violated: for example, we cannot check that a function fails fast as opposed to returning a garbage value on some illegal input, because all tests must follow the spec. If the precondition is one that the implementer can reasonably check, then it often makes sense to change the spec: remove the precondition, and instead specify the behavior in the postcondition. We’ll discuss two ways to do that below (exceptions and special results) and dive deeply into spec-writing decisions like this in the next reading.

## Specifications for mutating functions

我们在前面的文章（静态检查）中讨论了可变对象和不可变对象。但是到目前为止，我们的规范示例还没有说明如何在后置条件中描述副作用--可变对象或输入/输出状态的变化。

因此，下面的规范描述了一个改变对象的函数：
```
addAll(array1: Array<string>, array2: Array<string>): boolean
requires: array1 and array2 are not the same object
effects: modifies array1 by adding the elements of array2 to the end of it, and returns true if and only if array1 changed as a result of call
```
首先看看后置条件。它给出了两个约束：第一个告诉我们如何修改array1，第二个告诉我们如何确定返回值。

其次，看一下前提条件。它告诉我们数组1和数组2不能是同一个对象。如果你试图将一个数组的元素添加到数组本身，函数的行为是未定义的。我们可以很容易地想象为什么函数的实现者希望施加这个约束：它不可能排除函数的任何有用的应用，而且它使函数更容易实现。规范允许一个简单的实现，即从array2中取出一个元素添加到array1中，然后继续从array2中取出下一个元素，直到最后。

如果array1和array2是同一个数组，那么这个简单的算法将不会结束，如右边的快照图所示--或者实际上，当数组对象已经大到消耗了所有可用内存时，它将抛出一个内存错误。无论是无限循环还是崩溃，都是规范所允许的，因为它有一个前提条件。
![](ref/lect4/2023-07-18-15-23-21.png)
![](ref/lect4/2023-07-18-15-23-54.png)
![](ref/lect4/2023-07-18-15-24-01.png)
下面是另一个突变函数的示例：
```
sort(array: Array<string>): void
requires: nothing
effects: puts array in sorted order, i.e. array[i] ≤ array[j] for all 0 ≤ i < j < array.length

```
函数不改变参数的例子：
```
toLowerCase(array: Array<string>): Array<string>
requires: nothing
effects: returns a new array t, with same length as array, where t[i] = array[i].toLowerCase()
```
正如除非另有说明，否则null是隐式不允许的一样，程序员也认为除非另有说明，否则突变是不允许的。toLowerCase的规范可以明确指出 "数组没有被修改"，但是在后置条件中没有描述会发生突变的情况下，我们要求输入不会发生突变。

当在 TypeDoc 中编写规范时，影响参数的突变可以在相应的 @param 子句中描述：
```ts
/**
 * Sorts an array.
 * @param array - mutated into sorted order, so that
 *     array[i] ≤ array[j] for all 0 ≤ i < j < array.length.
 */
function sort(array: Array<string>): void { ... }
```

## Exception
既然我们已经编写了规范，并考虑了客户将如何使用我们的函数，那么让我们来讨论一下如何以一种避免错误和易于理解的方式来处理异常情况。

由于异常是函数的一种可能输出，因此必须在函数的后置条件中对其进行描述。可以在文档注释中使用 @throws 子句来记录异常。本节将讨论何时在规范中包含异常，何时不包含异常。
### Exceptions for signaling bugs
到目前为止，您可能已经在 Python 或 TypeScript 编程中看到过一些异常。例如，在Python中

* 当数组索引array[i]超出数组数组的有效范围时，会抛出IndexError。
* 当字典查询dict[key]没有找到匹配的键时，会抛出KeyError。
* TypeError会在各种动态类型错误时抛出，例如尝试调用None上的方法。
TypeScript也有用于动态类型错误的TypeError，但是大多数动态错误是用通用的Error类来提示的。

这类异常通常表示客户端或实现中的错误，异常抛出时显示的信息可以帮助您找到并修复错误。

提示错误的异常不属于函数的后置条件，因此它们不应出现在 @throws 中。例如，TypeError 通常不应在规范中提及。参数的类型是先决条件的一部分，这意味着如果这些类型被违反（例如，如果客户端设法传递了一个null），一个有效的实现可以自由地抛出TypeError而不需要任何警告。例如，本规范从未提及TypeError：
```ts
/**
 * @param array array of strings to convert to lower case
 * @returns new array t, same length as array, 
 *         where t[i] is array[i] converted to lowercase for all valid indices i
 */
function toLowerCase(array: Array<string>): Array<string>
```

### Exceptions for anticipated failures
异常不仅仅用于提示错误。异常可以用来提示预期的故障源，以便调用者能够捕获并响应它们。提示预期故障的异常应该用@throws子句来记录，指定故障发生的条件。例如
```ts
/**
 * Compute the integer square root.
 * @param x integer value to take square root of
 * @returns square root of x
 * @throws NotPerfectSquareError if x is not a perfect square
 */
function integerSquareRoot(x: number): number
```
这是新的addAll spec：
```ts
/**
 * If the array1 !== array2, adds the elements of array2 to the end of array1.
 * @returns true if array1 changed as a result
 * @throws AliasingError if array1 === array2
 */
function addAll(array1: Array<string>, array2: Array<string>): boolean
```
这是原始的addAll spec
```
addAll(array1: Array<string>, array2: Array<string>): boolean
requires: array1 and array2 are not the same object
effects: modifies array1 by adding the elements of array2 to the end of it, and returns true if and only if array1 changed as a result of call
```
在上面最初的addAll中，规范包含了array1 !== array2作为前提条件：如果违反了这个要求，函数的行为是未定义的。在这个版本的规范中，前置条件被移除，取而代之的是一个后置条件，它定义了当数组是不同对象时的行为（"adds the elements of array2 to the end of array1."和@returns子句），以及当它们是同一个对象时的行为（@throws子句）。

## Special results

异常是传达调用者不可能处理的问题的最佳方式，例如，由于调用者提供了非法输入：调用者或调用者有bug。

对于调用者应该准备处理的异常或特殊结果，不同的编程语言采取了不同的方法。例如，在Java语言中，实现者可以声明静态要求调用者处理的异常：如果调用者没有捕获exception或没有将其作为抛出异常添加到自己的规范中，编译器会给出一个静态错误。

TypeScript不提供异常处理的静态检查，所以使用异常来处理特殊情况的结果可能是bug的来源。例如，假设我们使用integerSquareRoot，输入来自文件或由用户输入，我们忽略了try......catch NotPerfectSquareError。一个无效的数字将使程序崩溃，此时最好产生一个清晰的错误信息或提示用户重试。但是TypeScript缺乏帮助我们提前发现问题的对exception的静态检查。

一个很好的替代方法是使函数的返回类型成为[联合类型](https://www.typescriptlang.org/docs/handbook/2/everyday-types.html#union-types)，这种类型的值集是由两个（或更多）其他类型定义的值集的联合：

```ts
/**
 * Compute the integer square root.
 * @param x integer value to take square root of
 * @returns square root of x if x is a perfect square, undefined otherwise
 */
function integerSquareRoot(x: number): number|undefined
```
undefined与null非常相似：在JavaScript中，已声明但未初始化的变量的值是undefined而不是null。如果没有严格的空值检查，任何变量都可以是undefined或null。如果进行了严格的检查，这两种情况都会被排除。

按照惯例，null表示 "没有值的值"，在任何时候都应避免使用。与此相反，undefined表示 "no value here at all"，只要开启了严格的null检查，我们就可以在联合类型中使用undefined来表示特殊情况的结果。在我们的示例中，试图使用更新的 integerSquareRoot 而不考虑它可能返回 undefined 的可能性在很多情况下是一个静态错误：

```ts
let twice = integerSquareRoot(input) * 2; // static error
if (integerSquareRoot(input) > 4) { ... } // static error
```

```ts
console.log('Your lucky number is:', integerSquareRoot(5)); // no error
```

### Missing Map keys
Python
```ts
zoo = { 'Tim': 'beaver' }
zoo['Tim']     # => 'beaver'
zoo['Bao Bao'] # => raises KeyError: 'Bao Bao'
```
Python 选择抛出异常，TS则不一样
```ts
const zoo = new Map([['Tim', 'beaver']]);
zoo.get('Tim');     // => 'beaver'
zoo.get('Bao Bao'); // => undefined
```
在Python中，后置条件指定了一个KeyError。在TypeScript中，对于值类型为V的Map，后置条件指定返回类型为V|undefined。

### Illegal array indices
```ts
const zoo = [ 'Tim' ];
for (let i = 0; i <= 1; i++) {
  let name: string = zoo[i];
  console.log(`Hello, ${name}!`);
}
// => prints:
//    Hello, Tim!
//    Hello, undefined!
```
哦，不。编译器允许将zoo[i]赋值给name，尽管name必须是字符串，而zoo[i]可能不是！这是静态类型系统中的另一个漏洞，它需要TypeScript的另一点配置：打开不检查索引访问将改变[...]索引操作符的返回类型，并使其与undefined联合。到目前为止，在6.102中这个选项是关闭的。我们将在以后的练习和问题集中打开它。一旦我们这样做了，我们就必须考虑到索引在这样的代码中可能是非法的，或者修改代码以避免使用索引。

[edit](https://www.typescriptlang.org/play?noUncheckedIndexedAccess=true#code/MYewdgzgLgBAXiEMC8MDaMDkAVAlgW0xgF0BuAKADMQAnGACgBsBTWXFGABlJnYB5UARh64A1KICUMAN7kYMFrDABDfMwBcMaDVxgA5hwQg0uMnJihIIFgDpGIPfQAGACWaN7AGhgASaSrUAXwBCJwkKQPIgA)

## Summary

规范是模块实现者和客户之间的一道重要防火墙。它使单独开发成为可能：客户可以自由地编写使用模块的代码，而无需查看其源代码；实现者可以自由地编写实现模块的代码，而无需知道它将如何被使用。

让我们回顾一下规范是如何帮助实现本课程的主要目标的：

* Safe from bugs. A good specification clearly documents the mutual assumptions that a client and implementer rely on. Bugs often come from disagreements at the interfaces, and the presence of a specification helps avoid those disagreements. Using machine-checked language features in your spec, like static typing and exceptions rather than just a human-readable comment, can reduce bugs still more.

* Easy to understand. A short, simple spec is easier to understand than the implementation itself, and saves other people from having to read the code.

* Ready for change. Specs establish contracts between different parts of your code, allowing those parts to change independently as long as they continue to satisfy the requirements of the contract.